In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
with open('vetting_playlist.json', 'r') as file:
    data1 = json.load(file)

In [3]:
#data1.keys()
#This is for someone else

In [4]:
file_paths = [
    'mpd.slice.15000-15999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_id": track['track_uri'],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
                # Add and remove for DBSCAN
                if len(all_data) >= 1000:
                    break
            if len(all_data) >= 1000:
                break
        if len(all_data) >= 1000:
            break
df = pd.DataFrame(all_data)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         1000 non-null   object
 1   position_in_playlist  1000 non-null   int64 
 2   track_id              1000 non-null   object
 3   track_name            1000 non-null   object
 4   artist_name           1000 non-null   object
 5   duration_ms           1000 non-null   int64 
 6   album_name            1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [6]:
# Count how many playlists each track appears in
track_counts = df.groupby(['track_id', 'track_name', 'artist_name']).size().reset_index(name='count')

In [7]:
# Normalize counts to a popularity score (1-10)
min_count = track_counts['count'].min()
max_count = track_counts['count'].max()
track_counts['popularity_score'] = (
    ((track_counts['count'] - min_count) / (max_count - min_count) * 9 + 1
).astype(int))

In [8]:
# Merge popularity scores back into the original DataFrame
df_with_popularity = df.merge(
    track_counts[['track_id', 'popularity_score']],
    on='track_id',
    how='left'
)

# Popularity Score

## What Does the Score Tell You?
#### High Popularity Score (8-10):

#### Tracks with high scores are included in many playlists, indicating they are widely popular or well-liked.

#### Medium Popularity Score (4-7):

#### Tracks with medium scores are moderately popular and appear in a reasonable number of playlists.

#### Low Popularity Score (1-3):

#### Tracks with low scores are less popular and appear in only a few playlists.

In [9]:
df_with_popularity['popularity_score']

0      1
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: popularity_score, Length: 1000, dtype: int32

In [10]:
df_with_popularity[df_with_popularity['popularity_score'] == 1]

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name,popularity_score
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham,1
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham,1
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons,1
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For,1
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP,1
...,...,...,...,...,...,...,...,...
995,house/edm,30,spotify:track:6GX1zjEDqeI6ogBtkFjleM,Perfect - Matoma Remix,One Direction,223826,Perfect - EP,1
996,house/edm,31,spotify:track:1bYuKo7Ph4qS5Klw9zzD74,Last Night Of Our Lives - Radio Edit,Manse,181500,Last Night Of Our Lives,1
997,house/edm,32,spotify:track:0tICYNayWWhH9GPeFrfjfD,This Girl (Kungs Vs. Cookin' On 3 Burners) - K...,Kungs,195546,Layers,1
998,house/edm,33,spotify:track:0NF4I7fY4eRKpDA6fPJkZT,Tell Me,Kaaze,173906,Tell Me,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         1000 non-null   object
 1   position_in_playlist  1000 non-null   int64 
 2   track_id              1000 non-null   object
 3   track_name            1000 non-null   object
 4   artist_name           1000 non-null   object
 5   duration_ms           1000 non-null   int64 
 6   album_name            1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [12]:
df.head()

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP


In [13]:
track_name = df.drop_duplicates(subset=['track_name'])

In [14]:
track_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 968 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         968 non-null    object
 1   position_in_playlist  968 non-null    int64 
 2   track_id              968 non-null    object
 3   track_name            968 non-null    object
 4   artist_name           968 non-null    object
 5   duration_ms           968 non-null    int64 
 6   album_name            968 non-null    object
dtypes: int64(2), object(5)
memory usage: 60.5+ KB


In [15]:
track_name.head(1000)

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP
...,...,...,...,...,...,...,...
995,house/edm,30,spotify:track:6GX1zjEDqeI6ogBtkFjleM,Perfect - Matoma Remix,One Direction,223826,Perfect - EP
996,house/edm,31,spotify:track:1bYuKo7Ph4qS5Klw9zzD74,Last Night Of Our Lives - Radio Edit,Manse,181500,Last Night Of Our Lives
997,house/edm,32,spotify:track:0tICYNayWWhH9GPeFrfjfD,This Girl (Kungs Vs. Cookin' On 3 Burners) - K...,Kungs,195546,Layers
998,house/edm,33,spotify:track:0NF4I7fY4eRKpDA6fPJkZT,Tell Me,Kaaze,173906,Tell Me


In [16]:
artists = df.drop_duplicates(subset=['artist_name'])

In [17]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         488 non-null    object
 1   position_in_playlist  488 non-null    int64 
 2   track_id              488 non-null    object
 3   track_name            488 non-null    object
 4   artist_name           488 non-null    object
 5   duration_ms           488 non-null    int64 
 6   album_name            488 non-null    object
dtypes: int64(2), object(5)
memory usage: 30.5+ KB


In [18]:
playlists = df.drop_duplicates(subset=['playlist_name'])

In [19]:
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 965
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         16 non-null     object
 1   position_in_playlist  16 non-null     int64 
 2   track_id              16 non-null     object
 3   track_name            16 non-null     object
 4   artist_name           16 non-null     object
 5   duration_ms           16 non-null     int64 
 6   album_name            16 non-null     object
dtypes: int64(2), object(5)
memory usage: 1.0+ KB


In [20]:
file_path = 'mpd.slice.15000-15999.json'  # Replace with your file path
with open(file_path, 'r') as file:
    data1 = json.load(file)

# Extract track information
tracks_data = []
for playlist in data1['playlists']:
    for track in playlist['tracks']:
        tracks_data.append({
            "playlist_name": playlist['name'],
            "track_name": track['track_name'],
            "artist_name": track['artist_name'],
            "track_id": track['track_uri'],  # Use 'track_uri' or 'track_id'
            "duration_ms": track['duration_ms']
        })

# Convert to a DataFrame
df1 = pd.DataFrame(tracks_data)